In [783]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [784]:
# Carregar a base de dados
df = pd.read_csv('Carseats.csv', sep=';', index_col=0)

df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,High
1,"9,5",138,73,11,276,120,Bad,42,17,Yes,Yes,Yes
2,"11,22",111,48,16,260,83,Good,65,10,Yes,Yes,Yes
3,"10,06",113,35,10,269,80,Medium,59,12,Yes,Yes,Yes
4,"7,4",117,100,4,466,97,Medium,55,14,Yes,Yes,No
5,"4,15",141,64,3,340,128,Bad,38,13,Yes,No,No


In [785]:
le_urban = LabelEncoder()
le_us = LabelEncoder()
le_high = LabelEncoder()
oe_shelve_loc = OrdinalEncoder(categories=[['Bad', 'Medium', 'Good']])
scaler = MinMaxScaler()

def preprocessing(data: pd.DataFrame) -> pd.DataFrame:
    df = data
    
    if not hasattr(le_urban, 'classes_'):
        df['Urban'] = le_urban.fit_transform(df['Urban'])
    else:
        df['Urban'] = le_urban.transform(df['Urban'])
        
    if not hasattr(le_us, 'classes_'):
        df['US'] = le_us.fit_transform(df['US'])
    else:
        df['US'] = le_us.transform(df['US'])
        
    if not hasattr(le_high, 'classes_'):
        df['High'] = le_high.fit_transform(df['High'])
    else:
        df['High'] = le_high.transform(df['High'])
    
    if not hasattr(oe_shelve_loc, 'categories_'):
        df['ShelveLoc'] = oe_shelve_loc.fit_transform(df[['ShelveLoc']])
    else:
        df['ShelveLoc'] = oe_shelve_loc.transform(df[['ShelveLoc']])
    
    df['Sales'] = df['Sales'].apply(lambda x: float(x.replace(',', '.')))

    if not hasattr(scaler, 'mean_'):
        df[df.columns] = scaler.fit_transform(df[df.columns])
    else:
        df[df.columns] = scaler.transform(df[df.columns])

    return df

df = preprocessing(df)

df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,High
1,0.583897,0.622449,0.525253,0.379310,0.533066,0.574850,0.0,0.309091,0.875,1.0,1.0,1.0
2,0.689613,0.346939,0.272727,0.551724,0.501002,0.353293,1.0,0.727273,0.000,1.0,1.0,1.0
3,0.618316,0.367347,0.141414,0.344828,0.519038,0.335329,0.5,0.618182,0.250,1.0,1.0,1.0
4,0.454825,0.408163,0.797980,0.137931,0.913828,0.437126,0.5,0.545455,0.500,1.0,1.0,0.0
5,0.255071,0.653061,0.434343,0.103448,0.661323,0.622754,0.0,0.236364,0.375,1.0,0.0,0.0


In [786]:
def remove_outliers(data: pd.DataFrame) -> pd.DataFrame:
    df = data
    
    for col in ['Sales', 'CompPrice', 'Price']:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
    
    return df

In [787]:
datasets = [
    {
        'remove_outliers': False,
        'train_size': 0.5,
    },
    {
        'remove_outliers': False,
        'train_size': 0.7
    },
    {
        'remove_outliers': True,
        'train_size': 0.5
    },
    {
        'remove_outliers': True,
        'train_size': 0.7
    }
]

In [788]:
np.random.seed(69)

for set in datasets:
    data = df

    if set['remove_outliers']:
        data = remove_outliers(data)

    X = data.drop('Sales', axis=1)
    y = data['Sales']
    
    set['X_train'], set['X_test'], set['y_train'], set['y_test'] = train_test_split(X, y, train_size=set['train_size'])

In [789]:
model_params = {
    'knn': {
        'model': KNeighborsRegressor(),
        'params': {
            'n_neighbors': [3, 5],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        }
    },
    'regression_tree': {
        'model': DecisionTreeRegressor(),
        'params': {
            'criterion': ['squared_error', 'absolute_error', 'poisson'],
            'max_depth': [None, 7, 10],
            'min_samples_leaf': [1, 2, 3]
        }
    },
    'linear_regression': {
        'model': LinearRegression(),
        'params': {
            'fit_intercept': [True, False],
            'positive': [True, False]
        }
    }
}

In [790]:
columns = pd.MultiIndex.from_tuples([
    ('Pré-Processamento', 'Outliers', 'manter'),
    ('Pré-Processamento', 'Outliers', 'remover'),

    ('Mineração de Dados', 'Conjunto de treino (%)', '50%'),
    ('Mineração de Dados', 'Conjunto de treino (%)', '70%'),

    ('Mineração de Dados', 'KNN', 'n_neighbors=3'),
    ('Mineração de Dados', 'KNN', 'n_neighbors=5'),
    ('Mineração de Dados', 'KNN', 'weights=uniform'),
    ('Mineração de Dados', 'KNN', 'weights=distance'),
    ('Mineração de Dados', 'KNN', 'algorithm=auto'),
    ('Mineração de Dados', 'KNN', 'algorithm=ball_tree'),
    ('Mineração de Dados', 'KNN', 'algorithm=kd_tree'),
    ('Mineração de Dados', 'KNN', 'algorithm=brute'),
    ('Mineração de Dados', 'KNN', 'metric=euclidean'),
    ('Mineração de Dados', 'KNN', 'metric=manhattan'),
    ('Mineração de Dados', 'KNN', 'metric=minkowski'),

    ('Mineração de Dados', 'Árvore de Regressão', 'criterion=squared_error'),
    ('Mineração de Dados', 'Árvore de Regressão', 'criterion=absolute_error'),
    ('Mineração de Dados', 'Árvore de Regressão', 'criterion=poisson'),
    ('Mineração de Dados', 'Árvore de Regressão', 'max_depth=None'),
    ('Mineração de Dados', 'Árvore de Regressão', 'max_depth=7'),
    ('Mineração de Dados', 'Árvore de Regressão', 'max_depth=10'),
    ('Mineração de Dados', 'Árvore de Regressão', 'min_samples_leaf=1'),
    ('Mineração de Dados', 'Árvore de Regressão', 'min_samples_leaf=2'),
    ('Mineração de Dados', 'Árvore de Regressão', 'min_samples_leaf=3'),

    ('Mineração de Dados', 'Regressão Linear', 'fit_intercept=True'),
    ('Mineração de Dados', 'Regressão Linear', 'fit_intercept=False'),
    ('Mineração de Dados', 'Regressão Linear', 'positive=True'),
    ('Mineração de Dados', 'Regressão Linear', 'positive=False'),

    ('Pós-Processamento', 'Medidas', 'SQE'),
    ('Pós-Processamento', 'Medidas', 'RMSE'),
    ('Pós-Processamento', 'Medidas', 'R2')
])

experiment = pd.DataFrame(columns=columns)
experiment

Empty DataFrame
Columns: [(Pré-Processamento, Outliers, manter), (Pré-Processamento, Outliers, remover), (Mineração de Dados, Conjunto de treino (%), 50%), (Mineração de Dados, Conjunto de treino (%), 70%), (Mineração de Dados, KNN, n_neighbors=3), (Mineração de Dados, KNN, n_neighbors=5), (Mineração de Dados, KNN, weights=uniform), (Mineração de Dados, KNN, weights=distance), (Mineração de Dados, KNN, algorithm=auto), (Mineração de Dados, KNN, algorithm=ball_tree), (Mineração de Dados, KNN, algorithm=kd_tree), (Mineração de Dados, KNN, algorithm=brute), (Mineração de Dados, KNN, metric=euclidean), (Mineração de Dados, KNN, metric=manhattan), (Mineração de Dados, KNN, metric=minkowski), (Mineração de Dados, Árvore de Regressão, criterion=squared_error), (Mineração de Dados, Árvore de Regressão, criterion=absolute_error), (Mineração de Dados, Árvore de Regressão, criterion=poisson), (Mineração de Dados, Árvore de Regressão, max_depth=None), (Mineração de Dados, Árvore de Regressão, max_depth=7), (Mineração de Dados, Árvore de Regressão, max_depth=10), (Mineração de Dados, Árvore de Regressão, min_samples_leaf=1), (Mineração de Dados, Árvore de Regressão, min_samples_leaf=2), (Mineração de Dados, Árvore de Regressão, min_samples_leaf=3), (Mineração de Dados, Regressão Linear, fit_intercept=True), (Mineração de Dados, Regressão Linear, fit_intercept=False), (Mineração de Dados, Regressão Linear, positive=True), (Mineração de Dados, Regressão Linear, positive=False), (Pós-Processamento, Medidas, SQE), (Pós-Processamento, Medidas, RMSE), (Pós-Processamento, Medidas, R2)]
Index: []

[0 rows x 31 columns]

In [791]:
def sqe(y_true, y_pred):
    return np.sum((y_true - y_pred) ** 2)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scorers = {
    'SQE': make_scorer(sqe, greater_is_better=True),
    'RMSE': make_scorer(rmse, greater_is_better=True),
    'R2': make_scorer(r2_score, greater_is_better=True)
}

In [792]:
scores = []
i = 0

for set in datasets:
    X_train = set['X_train']
    X_test = set['X_test']
    y_train = set['y_train']
    y_test = set['y_test']

    for model_name, mp in model_params.items():
        clf = GridSearchCV(mp['model'], mp['params'], cv=5, scoring=scorers, return_train_score=False, refit='R2')
        clf.fit(X_train, y_train)
        results = pd.DataFrame(clf.cv_results_)[['params', 'mean_test_SQE', 'mean_test_RMSE', 'mean_test_R2']]

        for _, metrics in results.iterrows():
            row = pd.Series(index=columns, dtype=str)
            row[:] = ' '
        
            if set['remove_outliers']:
                row[('Pré-Processamento', 'Outliers', 'remover')] = 'x'
            else:
                row[('Pré-Processamento', 'Outliers', 'manter')] = 'x'

            if set['train_size'] == 0.5:
                row[('Mineração de Dados', 'Conjunto de treino (%)', '50%')] = 'x'
            else:
                row[('Mineração de Dados', 'Conjunto de treino (%)', '70%')] = 'x'

            match model_name:
                case 'knn':
                    pass
                case 'regression_tree':
                    pass
                case 'linear_regression':
                    pass

            row[('Pós-Processamento', 'Medidas', 'SQE')] = metrics['mean_test_SQE']
            row[('Pós-Processamento', 'Medidas', 'RMSE')] = metrics['mean_test_RMSE']
            row[('Pós-Processamento', 'Medidas', 'R2')] = metrics['mean_test_R2']

        experiment.loc[i] = row

        i += 1

experiment.head()

Pré-Processamento             Mineração de Dados                    \
           Outliers         Conjunto de treino (%)               KNN   
             manter remover                    50% 70% n_neighbors=3   
0                 x                              x                     
1                 x                              x                     
2                 x                              x                     
3                 x                                  x                 
4                 x                                  x                 

                                                                 \
                                                                  
  n_neighbors=5 weights=uniform weights=distance algorithm=auto   
0                                                                 
1                                                                 
2                                                                 
3                                                                 
4                                                                 

                       ...                                         \
                       ... Árvore de Regressão                      
  algorithm=ball_tree  ...  min_samples_leaf=1 min_samples_leaf=2   
0                      ...                                          
1                      ...                                          
2                      ...                                          
3                      ...                                          
4                      ...                                          

                                                                           \
                       Regressão Linear                                     
  min_samples_leaf=3 fit_intercept=True fit_intercept=False positive=True   
0                                                                           
1                                                                           
2                                                                           
3                                                                           
4                                                                           

                 Pós-Processamento                      
                           Medidas                      
  positive=False               SQE      RMSE        R2  
0                         0.406808  0.100208  0.657281  
1                         0.615921  0.123929  0.451012  
2                         0.400133  0.099786  0.659841  
3                         0.651613  0.107826  0.588955  
4                         0.894194  0.126216  0.439647  

[5 rows x 31 columns]

In [793]:
experiment.to_excel('experimento.xlsx')